**CSI 5139 Assignment 3**

Section 2 of this assignment explores transfer learning with VGG16 model for classification task of 68 textures.

# Setup

In [1]:
import os
from shutil import copy

import tensorflow as tf
import tensorflow.keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, MaxPooling2D, Conv2D, Input
from tensorflow.keras.optimizers import RMSprop, Adagrad, Adam, SGD
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import load_model, Model

import matplotlib
import matplotlib.pyplot as plt
import numpy as np

from sklearn.metrics import confusion_matrix

from pprint import pprint

C:\Users\Sophie\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# VGG-16

In [66]:
datagen = ImageDataGenerator(rescale=1./255)


train_batchsize = 32 # 160 images per 68 classes
test_batchsize = 4 # 20 images per 68 classes

train_generator = datagen.flow_from_directory(
    directory=r"C:/Users/Sophie/Documents/GitHub/CSI5139/csi5139_a2-Copy/Outex-TC-00030/train/",
    target_size=(128, 128),
    color_mode="rgb",
    batch_size=train_batchsize,
    class_mode="categorical",
    shuffle=True,
    seed=42
)

test_generator = datagen.flow_from_directory(
    directory=r"C:/Users/Sophie/Documents/GitHub/CSI5139/csi5139_a2-Copy/Outex-TC-00030/test/",
    target_size=(128, 128),
    color_mode="rgb",
    batch_size=test_batchsize,
    class_mode="categorical",
    shuffle=True,
    seed=42
)

print("Train generator length: ", len(train_generator))
print("Test generator length: ", len(test_generator))
print("Target size: ", train_generator.target_size)
#print("Generator indices: ", train_generator.class_indices)
#print("Generator filenames:", train_generator.filenames[:10])

Found 10880 images belonging to 68 classes.
Found 1360 images belonging to 68 classes.
Train generator length:  340
Test generator length:  340
Target size:  (128, 128)


#### Model

In [93]:
epochs = 3

vgg_conv = VGG16(weights='imagenet', include_top=False, input_shape=(128, 128, 3))
vgg_conv.summary()

for layer in vgg_conv.layers:
    layer.trainable = False

# last layers output
x = vgg_conv.layers[-1].output
# Flatten as before
x = Flatten()(x)
x = Dense(512, activation='relu')(x)
x = Dense(68, activation='softmax')(x)

vgg_conv_transfer = Model(inputs=vgg_conv.input, outputs=x)

for layer in vgg_conv_transfer.layers:
    print(layer, layer.trainable)

vgg_conv_transfer.summary()

rmsprop = RMSprop()
sgd = SGD()

cp_callback = tensorflow.keras.callbacks.ModelCheckpoint(filepath="./runs/vgg16.ckpt", save_weights_only=True, verbose=1, save_best_only=True)

vgg_conv_transfer.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_36 (InputLayer)        (None, 128, 128, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 128, 128, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 128, 128, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 64, 64, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 64, 64, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 64, 64, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 32, 32, 128)       0         
__________

#### Initial training

In [76]:
hist_vgg16 = vgg_conv_transfer.fit_generator(
        train_generator,
        steps_per_epoch=(train_generator.samples/train_generator.batch_size),
        epochs=epochs,
        validation_data=test_generator,
        validation_steps=(test_generator.samples/test_generator.batch_size),
        verbose=1,
        callbacks = [cp_callback])

Epoch 1/3
339/340 [============================>.] - ETA: 6s - loss: 3.1522 - acc: 0.6645 
Epoch 00001: val_loss improved from inf to 1.40600, saving model to ./runs/vgg16.ckpt
340/340 [==============================] - 2321s 7s/step - loss: 3.1468 - acc: 0.6649 - val_loss: 1.4060 - val_acc: 0.7654
Epoch 2/3
339/340 [============================>.] - ETA: 5s - loss: 0.5326 - acc: 0.8361 
Epoch 00002: val_loss improved from 1.40600 to 0.38671, saving model to ./runs/vgg16.ckpt
340/340 [==============================] - 1985s 6s/step - loss: 0.5322 - acc: 0.8362 - val_loss: 0.3867 - val_acc: 0.8449
Epoch 3/3
339/340 [============================>.] - ETA: 5s - loss: 0.3703 - acc: 0.8604 
Epoch 00003: val_loss did not improve from 0.38671
340/340 [==============================] - 2048s 6s/step - loss: 0.3704 - acc: 0.8605 - val_loss: 0.3881 - val_acc: 0.8544


In [77]:
# save history
hist_vgg16 = hist_vgg16.history
with open('./runs/vgg16.hist', 'w', encoding="utf-8") as fout:
    pprint(hist_vgg16, fout)

#### Train more layers

In [108]:
# continue training model
vgg_conv_transfer.load_weights("./runs/vgg16.ckpt")

# higher level layers set trainable to true
vgg_conv_transfer.layers[-4].trainable = True
vgg_conv_transfer.layers[-5].trainable = True
vgg_conv_transfer.layers[-6].trainable = True
vgg_conv_transfer.layers[-7].trainable = True

for layer in vgg_conv_transfer.layers:
    print(layer, layer.trainable)

sgd = SGD(0.0005)
#rmsprop = RMSprop(1e-4)
vgg_conv_transfer.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])

<tensorflow.python.keras.engine.input_layer.InputLayer object at 0x000001F8E75A6198> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x000001F8E75A6630> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x000001F8E92A0978> False
<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x000001F8E75A6400> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x000001F8E75A6E48> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x000001F8E92BD5F8> False
<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x000001F8E9264A58> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x000001F8E9268E10> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x000001F8E9307A20> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x000001F8E933F7B8> False
<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x000001F8E93704A8> False
<tensorflow.python.

In [109]:
hist_vgg16 = vgg_conv_transfer.fit_generator(
        train_generator,
        steps_per_epoch=(train_generator.samples/train_generator.batch_size),
        epochs=epochs,
        validation_data=test_generator,
        validation_steps=(test_generator.samples/test_generator.batch_size),
        verbose=1,
        callbacks = [cp_callback])

Epoch 1/3
339/340 [============================>.] - ETA: 6s - loss: 0.2681 - acc: 0.8949 
Epoch 00001: val_loss improved from inf to 0.28065, saving model to ./runs/vgg16.ckpt
340/340 [==============================] - 2688s 8s/step - loss: 0.2679 - acc: 0.8950 - val_loss: 0.2806 - val_acc: 0.8934
Epoch 2/3
339/340 [============================>.] - ETA: 10s - loss: 0.2219 - acc: 0.9137
Epoch 00002: val_loss improved from 0.28065 to 0.27699, saving model to ./runs/vgg16.ckpt
340/340 [==============================] - 4054s 12s/step - loss: 0.2216 - acc: 0.9138 - val_loss: 0.2770 - val_acc: 0.8941
Epoch 3/3
339/340 [============================>.] - ETA: 10s - loss: 0.1934 - acc: 0.9247
Epoch 00003: val_loss improved from 0.27699 to 0.26269, saving model to ./runs/vgg16.ckpt
340/340 [==============================] - 4042s 12s/step - loss: 0.1933 - acc: 0.9247 - val_loss: 0.2627 - val_acc: 0.8919
